In [ ]:
import sqlparse
from sqlparse.sql import IdentifierList, Identifier
from sqlparse.tokens import Keyword, DML
from query_representation.utils import *

#from moz_sql_parser import parse
#from mo_sql_parsing import parse as parse
import os
import pandas as pd
from collections import defaultdict

from IPython.display import HTML
from IPython.display import display_html

In [ ]:
QDIR = "./queries/tpcds/all"
#QDIR = "queries/job/all_job/sqls/all_job"
#QDIR = "ceb-unique-sqls"
TPCDS_TABLES="./queries/tpcds/tpcds.sql"

In [ ]:

with open(TPCDS_TABLES) as f:
    tdata = f.readlines()
#print(tdata)

COLS_TO_TABLES = {}

for li, line in enumerate(tdata):
    #print(line)
    if "create table" in line:
        #print(line)
        vals = line.split(" ")
        #print("table: ", vals[2])
        table = vals[2].replace("\n", "")
        
        for i in range(1000):
            cline = tdata[li+i]
            cline = cline.lstrip()
            #print(cline)
                
            if "create table" in cline or "primary key" in cline:
                continue
            vals = cline.split(" ")
            col = vals[0]
            
            if not (")" in col or "(" in col or col == ""):
                if col in COLS_TO_TABLES:
                    print("?>>???")
                    print(col)
                assert col not in COLS_TO_TABLES   
                COLS_TO_TABLES[col] = table
            
            if ")" in cline and "," not in cline:
                break
        

In [ ]:
#print(COLS_TO_TABLES)

In [ ]:
#"catery" in COLS_TO_TABLES
for k in COLS_TO_TABLES:
    if "cat" in k:
        print(k)

In [ ]:
FNS = os.listdir(QDIR)

In [ ]:
print(len(FNS))
print(FNS[19])

In [ ]:
sqls = []
for fn in FNS:
    if ".sql" not in fn:
        continue
    if "all" in fn:
        continue
    fn = os.path.join(QDIR, fn)
    with open(fn, "r") as f:
        sql = f.read()
    #if "like" in sql.lower():
        #print(sql)
    sql = sql.replace("IN", "in")
    sql = sql.replace("In", "in")
    sql = sql.replace("ILIKE", "like")
    sqls.append(sql)

In [ ]:
print(len(sqls))

In [ ]:
# for si, sql in enumerate(sqls):
#     if "like" in sql.lower():
#         print(si)
#     if "q91" in sql.lower():
#         print(si)

In [ ]:
#sql = sqls[40]
sql = sqls[78]
# sql = sql.replace("IN", "in")
# sql = sql.replace("In", "in")
# sql = sql.replace("ILIKE", "like")

In [ ]:
print(sql)

In [ ]:
# def get_cols_from_tokens(tokens):
#     cols = []
#     for token in tokens:
#         if isinstance(token, sqlparse.sql.Identifier):
#             cols.append(str(token.value))
#     return cols

def get_function_cols(func_token):
    cols = []
    for param in func_token.get_parameters():
        if isinstance(param, sqlparse.sql.Identifier):
            cols.append(param.value)
    return cols

def parse_sql_preds(sql):
    def get_table_col(left):
        if "." in left:
            tablename = left[0:left.find(".")]
            col = left[left.find(".")+1:]
        else:
            tablename = "X"
            col = left
        return tablename, col
        
    def parse_where(wtokens):
        curwhere = {}
        for idx, token in enumerate(wtokens):
            if "where" in str(type(token)).lower():
                parse_where(token)
                continue
            
            if isinstance(token, sqlparse.sql.Parenthesis):
                assert hasattr(token, "tokens")
                #parse_recursive(token)
                parse_where(token)
                continue
            
            if "comparison" in str(type(token)).lower():
                #print(token.value)
                assert hasattr(token, "tokens")
                left = None
                op = None
                right = None
                valtypes = []
        
                for ctoken in token:
                    if ctoken.value.strip() == "":
                        continue
                    assert right is None
            
                    if left is None:
                        left = ctoken
                        valtypes.append(type(ctoken))
                        continue
                    if op is None:
                        op = ctoken
                        continue
                        
                    if right is None:
                        right = ctoken
                        continue
                    
                if isinstance(left, sqlparse.sql.Identifier):
                    #left = left.value
                    tablename, col = get_table_col(left.value)  
                elif isinstance(left, sqlparse.sql.Function):
                    tablename = "function"
                    allcols = get_function_cols(left)
                    #print("Num function columns: ", len(allcols))
                    col = ",".join(allcols)
                elif isinstance(left, sqlparse.sql.Parenthesis):
                    assert hasattr(left, "tokens")
                    parse_where(left)
                    tablename = "function"
                    col = "function"
                elif isinstance(left, sqlparse.sql.Operation):
                    tablename = "function"
                    col = "function"
                else:
                    # FIXME: handle these better
                    if isinstance(right, sqlparse.sql.Identifier):
                        tablename, col = get_table_col(right.value)
                        tmp1 = right
                        right = left
                        left = tmp1
                        # FIXME: need to change other stuff, like operator type too
                    else:
                        assert False
                    
                if "catery" in col:
                    col = col.replace("catery", "category")
                    
                if isinstance(right, sqlparse.sql.Identifier):
                    #print("Join?")
                    continue
                
                if isinstance(right, sqlparse.sql.Parenthesis):
                    assert hasattr(right, "tokens")
                    parse_recursive(right)
                
                # TODO: need a better way to find joins
                if not "token" in str(type(right)).lower():
                    if not "select" in right.value and \
                        op.value == "=" and \
                        "." in right.value:
                        # potential join
                        continue
                
                op = op.value.lower()
                right = right.value
                op = op.lower()
                
                allpreddata["left"].append(left.value)
                allpreddata["table"].append(tablename)   
                allpreddata["col"].append(col)
                allpreddata["op"].append(op)
                allpreddata["val"].append(str(right))
            
            #### TODO:
            if "between" in str(token):
                left = wtokens[idx-2]
                
                if isinstance(left, sqlparse.sql.Identifier):
                    tablename, col = get_table_col(left.value)
                    #print(tablename, col)
                elif isinstance(left, sqlparse.sql.Parenthesis):
                    tablename = "betweenX"
                    col = "func"
                else:
                    tablename = "betweenX"
                    col = "unk"
                    #assert False
                
                allpreddata["left"].append(left.value)
                allpreddata["table"].append(tablename)
                allpreddata["col"].append(col)
                allpreddata["op"].append("between")
                allpreddata["val"].append(-1)
                
#                 left = wtokens[idx+2]
#                 right = wtokens[idx+6]
# FIXME: general alternative
#                 for ci in range(-2,7,1):
#                     print(ci)
#                     print(wtokens[idx+ci])
            
    def parse_recursive(tokens):
        for idx, token in enumerate(tokens):
            if hasattr(token, "tokens"):
                parse_recursive(token)
            
            # original code
            if "where" in str(type(token)).lower():
                parse_where(token)
            
            if token.value.lower() in ["having", "group by"]:
                # TODO: parse
                index, token2 = tokens.token_next(idx)
        
    allpreddata = defaultdict(list)
    allpreds = []
    
    if sql.strip() == "":
        return pd.DataFrame(allpreddata)

    parsed = sqlparse.parse(sql)
    assert len(parsed) == 1
    parse_recursive(parsed[0])
    #return allpreds
    df = pd.DataFrame(allpreddata)
    return df

In [ ]:
#print(sql)

In [ ]:
preds = parse_sql_preds(sql)
preds

In [ ]:
# #print(preds)
# p0 = preds[0]

# mergedd = {}
# for p in preds:
#     for table,colvals in p.items():
#         if table not in mergedd:
#             mergedd[table] = {}
#         for col,opvals in colvals.items():
#             if col not in mergedd[table]:
#                 mergedd[table][col] = {}
#             for op,vals in opvals.items():
#                 if op not in mergedd[table][col]:
#                     mergedd[table][col][op] = set()
#                 for cval in vals:
#                     mergedd[table][col][op].add(cval)
                

# #print(mergedd)
# for table,vals in mergedd.items():
#     print(table)
#     print(vals)
#     print("***********")

# Q87 very hard to parse;
### TODO: maybe queries w/ multiple subqueries, we just try to get each select block and parse them individually

In [ ]:
sql4 = """-- q87


select count(*) 
from (select distinct c_last_name, c_first_name, d_date
       from store_sales, date_dim, customer  -- skan_memo_stash_87
       where store_sales.ss_sold_date_sk = date_dim.d_date_sk
         and store_sales.ss_customer_sk = customer.c_customer_sk
         and d_month_seq between 1212 and 1212+11)
       except
      (select distinct c_last_name, c_first_name, d_date
       from catalog_sales, date_dim, customer
       where catalog_sales.cs_sold_date_sk = date_dim.d_date_sk
         and catalog_sales.cs_bill_customer_sk = customer.c_customer_sk
         and d_month_seq between 1212 and 1212+11)
       except
      (select distinct c_last_name, c_first_name, d_date
       from web_sales, date_dim, customer
       where web_sales.ws_sold_date_sk = date_dim.d_date_sk
         and web_sales.ws_bill_customer_sk = customer.c_customer_sk
         and d_month_seq between 1212 and 1212+11)
cool_cust

"""

# sql4 = """-- q87


# select count(*) 
# from ((select distinct c_last_name, c_first_name, d_date
#        from store_sales, date_dim, customer  -- skan_memo_stash_87
#        where store_sales.ss_sold_date_sk = date_dim.d_date_sk
#          and store_sales.ss_customer_sk = customer.c_customer_sk
#          and d_month_seq between 1212 and 1212+11)
#        except
#       (select distinct c_last_name, c_first_name, d_date
#        from catalog_sales, date_dim, customer
#        where catalog_sales.cs_sold_date_sk = date_dim.d_date_sk
#          and catalog_sales.cs_bill_customer_sk = customer.c_customer_sk
#          and d_month_seq between 1212 and 1212+11))
# cool_cust

# """

# sql4 = """-- q87


# select count(*) 
# from (select distinct c_last_name, c_first_name, d_date
#        from store_sales, date_dim, customer  -- skan_memo_stash_87
#        where store_sales.ss_sold_date_sk = date_dim.d_date_sk
#          and store_sales.ss_customer_sk = customer.c_customer_sk
#          and d_month_seq between 1212 and 1212+11)
# cool_cust
# """
parse_sql_preds(sql4)

# Fix for Q40, change order of from tables

In [ ]:

sql3 = """select top 100 
   w_state
  ,i_item_id
  ,sum(case when (cast(d_date as date) < cast ('1998-04-08' as date)) 
        then cs_sales_price - coalesce(cr_refunded_cash,0) else 0 end) as sales_before
  ,sum(case when (cast(d_date as date) >= cast ('1998-04-08' as date)) 
        then cs_sales_price - coalesce(cr_refunded_cash,0) else 0 end) as sales_after
 from
   warehouse  -- skan_memo_stash_40
  ,item
  ,date_dim
  ,catalog_sales left outer join catalog_returns on
       (cs_order_number = cr_order_number 
        and cs_item_sk = cr_item_sk)

 where
     i_current_price between 0.99 and 1.49
 and i_item_sk          = cs_item_sk
 and cs_warehouse_sk    = w_warehouse_sk 
 and cs_sold_date_sk    = d_date_sk
 and d_date >= dateadd(dd, -30, '1998-04-08')
 and d_date <= dateadd(dd, 30, '1998-04-08') 
 group by
    w_state,i_item_id
 order by w_state,i_item_id"""
parse_sql_preds(sql3)

# replace intersect by union seems to work for now;

In [ ]:
sql2 = """select top 100 count(*) from (
    select distinct c_last_name, c_first_name, d_date
    from store_sales, date_dim, customer -- skan_memo_stash_38
          where store_sales.ss_sold_date_sk = date_dim.d_date_sk
      and store_sales.ss_customer_sk = customer.c_customer_sk
      and d_month_seq between 1212 and 1212 + 11) test"""
sql2 = """select top 100 count(*) from (
    select distinct c_last_name, c_first_name, d_date
    from store_sales, date_dim, customer -- skan_memo_stash_38
          where store_sales.ss_sold_date_sk = date_dim.d_date_sk
      and store_sales.ss_customer_sk = customer.c_customer_sk
      and d_month_seq between 1212 and 1212 + 11
  union
    select distinct c_last_name, c_first_name, d_date
    from catalog_sales, date_dim, customer
          where catalog_sales.cs_sold_date_sk = date_dim.d_date_sk
      and catalog_sales.cs_bill_customer_sk = customer.c_customer_sk
      and d_month_seq between 1212 and 1212 + 11) tmp"""
parse_sql_preds(sql2)

In [ ]:
# def parse_cont_vals(ops, vals, col_names):
    
#     # values to return: one for each column
#     ## type: range, lt, gt
#     ## dtype:
#     ## range: None, or actual value
#     ret = {}
#     cont_idxs = []
    
#     cur_comb_op = ""
#     for oi, op in enumerate(ops):
#         if op == "And" or op == "Or":
#             cur_comb_op = op
        
#         if ">" in op or "<" in op:
#             cont_idxs.append(oi)
#             col = col_names[oi]
            
#             col = col_names[oi]
# #             val = vals[oi]
# #             if isinstance(val, list):
# #                 if len(val) == 0:
# #                     continue
# #                 assert len(val) == 1
# #                 val = val[0]
            
# #             val = val.replace("@", "")
# #             val = val.replace("\"", "")
# #             val = val.replace("\'", "")
# #             dtype, pval = get_cont_dtype(val)
            
#             if col not in ret:
#                 ret[col] = {}
#             ret[col]["dtype"] = dtype
#             if pval is not None and ">" in op:
#                 ret[col]["gt"] = pval
#             elif pval is not None and "<" in op:
#                 ret[col]["lt"] = pval
            
#             ret[col]["comb_op"] = cur_comb_op
    
#     for col in ret:
#         if "comb_op" in ret[col] and ret[col]["comb_op"] == "Or":
#             ctype = "discont"
        
#         elif "lt" in ret[col] and "gt" in ret[col]:
#             ctype = "range"
#             #assert ret[col]["comb_op"] == "And"
#             if "comb_op" not in ret[col]:
#                 print(vals)
#                 print(ops)

#             elif ret[col]["comb_op"] != "And":
#                 print(vals)
#                 print(ops)
            
#         elif "lt" in ret[col]:
#             ctype = "lt"
#         elif "gt" in ret[col]:
#             ctype = "gt"
#         else:
#             ctype = "other"
        
#         if "lt" in ret[col] and "gt" in ret[col]:
#             crange = ret[col]["lt"] - ret[col]["gt"]
#             if crange < 0:
#                 assert ret[col]["comb_op"] == "Or"
#             else:
#                 ret[col]["range"] = crange
        
#         ret[col]["cont_type"] = ctype

#     return ret

def is_num(val):
    try:
        float(val)
        return True
    except:
        return False
    
def is_a_date(val):
    
    val = val.replace("'", "")
    if "date" in val:
        return True
    
    elif is_num(val):
        num = float(val)
        if num > 1900 and num < 2010:
            return True
        
    elif "+" in val:
        cvals = val[0:val.find("+")]
        if is_num(cvals):
            num = float(cvals)
            if num > 1900 and num < 2010:
                return True
            
    elif "-" in val:
#         if val.count("-") == 2:
#             print(val)
        cvals = val[0:val.find("-")]
        if is_num(cvals):
            num = float(cvals)
            if num > 1900 and num < 2010:
                return True
    
    return False


import enchant
enchantD = enchant.Dict("en_US")
import re
#import nltk
#nltk.download('words')
#from nltk.corpus import words as nltkwords

URLPAT = "((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*"
def get_like_type(op, vals):
    likekind = "-1"
    likedtype = "-1"
    
    if "like" not in op.lower():
        return likedtype, likekind
    
    vals = vals.replace("'", "")
    if vals.count("%") == 0:
        likekind = "no%"
    elif vals.count("%") == 2:
        likekind = "contains"
    elif vals.count("%") == 1:
        if vals[0] == "%":
            likekind = "ends"
        elif vals[-1] == "%":
            likekind = "starts"
        else:
            assert False
    else:
        nlikes = vals.count("%")
        likekind = str(nlikes) + "%"
    
    opval = vals.replace("%", "")
    
    if is_num(opval):
        likedtype = "num"
    elif len(opval) <= 2:
        likedtype = "short"
    elif opval[0] == ".":
        likedtype = "extension"
    elif re.match(URLPAT, opval) is not None:
        if ("cosmos" in opval or "adl" in opval) and "/" in opval:
            likedtype = "path"
        else:
            likedtype = "url"
            
    elif opval.count("/") >= 2 or opval.count("\\") >= 2:
        likedtype = "path"
        
    elif enchantD.check(opval):
        likedtype = "word"

    elif opval.count("-") >= 2 or \
            opval.count(":") >= 2:
        likedtype = "serial"
    elif "-" in opval or "_" in opval or " " in opval or "," in opval or ":" in opval:
        if "-" in opval:       
            allvals = opval.split("-")
        elif "_" in opval:
            allvals = opval.split("_")
        elif " " in opval:
            allvals = opval.split(" ")
        elif "," in opval:
            allvals = opval.split(",")
        elif ":" in opval:
            allvals = opval.split(":")
            
        validwords = 0
        for v1 in allvals:
            if v1 == "":
                continue
            if enchantD.check(v1):
                validwords += 1
        if validwords >= 2:
            likedtype = "words"
    elif "0x" in opval:
        likedtype = "hex"
    else:
        validwords = 0
        start = 0
        prevstart = 0
        for oi,_ in enumerate(opval):
            cword = opval[start:oi+1]
            pword = opval[prevstart:oi+1]
            if enchantD.check(cword) and len(cword) >= 3:
                validwords += 1
                prevstart = start
                start = oi+1
            
            # extend previous word
            elif enchantD.check(pword) and len(pword) >= 3:
                start = oi+1
                
        if validwords >= 2:
            likedtype = "words"
        else:
            likedtype = "unknown"
    
    return likedtype, likekind

def get_discrete_type(op, vals):
    dtype = "-1"
    dkind = "-1"
    #op = op.lower()
    assert op != ""
    if op in ["=", "!=", "<>", "in"]:
        if op == "=":
            dkind = "eq"
        elif op in ["!=", "<>"]:
            print("neq!!")
            dkind = "neq"
        else:
            dkind = "in"
            
        if "select" in vals:
            dtype = "sql"
        else: 
            if "(" in vals:
                vals = vals.replace("(", "")
                vals = vals.replace(")", "")
                vals = vals.split(",")
                vals = vals[0]
            
            if is_a_date(vals):
                dtype = "date"
            elif is_num(vals):
                dtype = "num"
            else:
                dtype = "string"
                
    
        
    return dtype, dkind

In [ ]:
from collections import defaultdict

allinpdata = defaultdict(list)
allpreds = []

for si, sql in enumerate(sqls):
    # empty sqls
    if si in [78, 85]:
        continue
    print("Query: ", si)
    alldfdata = defaultdict(list)
    
    numwheres = sql.lower().count("where")
    preds = parse_sql_preds(sql)
    olen = len(preds)
    preds = preds.drop_duplicates()
    nlen = len(preds)
    
    preds["qid"] = si
    
    curcols = []
    curops = []
    for idx, row in preds.iterrows():
        curcols.append(row["col"])
        curops.append(row["op"])
        
        if row["col"] in COLS_TO_TABLES:
            tab = COLS_TO_TABLES[row["col"]]
            col = row["col"]
        elif row["table"] != "X":
            tab = row["table"]
            col = row["col"]
        else:
            tab = row["table"]
        
        op = row["op"]
        alldfdata["table"].append(tab)
        alldfdata["col"].append(col)
        dtype,dkind = get_discrete_type(op, row["val"])
        alldfdata["dtype"].append(dtype)
        alldfdata["dkind"].append(dkind)
        
        likedtype,likekind = get_like_type(op, row["val"])
        alldfdata["likedtype"].append(likedtype)
        alldfdata["likekind"].append(likekind)
        
        if op in ["=", "!="]:
            alldfdata["discrete_ops"].append(1)
            alldfdata["in_ops"].append(0)
            allinpdata["col"].append(row["col"])
            allinpdata["val"].append(row["val"])
            
        elif op == "in":
            alldfdata["discrete_ops"].append(1)
            alldfdata["in_ops"].append(1)
            if "select" in row["val"]:
                pass
            else:
                cvals = row["val"]
                cvals = cvals.replace("(", "")
                cvals = cvals.replace(")", "")
                cvals = cvals.split(",")
                for cval in cvals:
                    allinpdata["col"].append(row["col"])
                    allinpdata["val"].append(cval)
        else:
            alldfdata["discrete_ops"].append(0)
            alldfdata["in_ops"].append(0)

        if op in [">", "<", ">=", "<=", "between"]:
            alldfdata["cont_ops"].append(1)
        else:
            alldfdata["cont_ops"].append(0)

        if op == "like":
            alldfdata["like_ops"].append(1)
        else:
            alldfdata["like_ops"].append(0)
    
    if len(preds) == 0:
        continue
    
    tmpdf = pd.DataFrame(alldfdata)
    assert len(tmpdf) == len(preds)
#     preds["like_ops"] = tmpdf["like_ops"]
#     preds["cont_ops"] = tmpdf["cont_ops"]
#     preds["discrete_ops"] = tmpdf["discrete_ops"]
#     preds["in_ops"] = tmpdf["in_ops"]
#     preds["dtype"] = tmpdf["dtype"]
#     preds["dkind"] = tmpdf["dkind"]
#     preds["likekind"] = tmpdf["likekind"]
    for k in tmpdf.keys():
        preds[k] = tmpdf[k]
        
    allpreds.append(preds)

In [ ]:
# for k,v in alldfdata.items():
#     print(k, len(v))

In [ ]:
df = pd.concat(allpreds)

In [ ]:
df.head(6)

In [ ]:
print(df.keys())
print(len(df))

In [ ]:
#df.to_csv("ceb_queries_df.csv", index=False)
df.to_csv("tpcds_queries_df.csv", index=False)

In [ ]:
# df2 = pd.read_csv("job_queries_df.csv")
# df2.head(5)

In [ ]:
HTML(df[["like_ops", "discrete_ops", "cont_ops", "in_ops"]].\
     describe(percentiles=[0.9,0.99]).reset_index().to_html(index=False))

In [ ]:
#print(set(df["dkind"]))

In [ ]:
discdf = df[df.discrete_ops == 1]
discdf.groupby(["dkind"]).count()["discrete_ops"].reset_index().sort_values(by="discrete_ops")

In [ ]:
discdf.groupby(["dtype"]).count()["discrete_ops"].reset_index().sort_values(by="discrete_ops", ascending=False)

In [ ]:
ldf = df[df.like_ops == 1]
ldf.groupby("likedtype").count()["discrete_ops"].reset_index().sort_values(by="discrete_ops", ascending=False)

In [ ]:
ldf.groupby("likekind").count()["discrete_ops"].reset_index().sort_values(by="discrete_ops", ascending=False)

In [ ]:
inpdata = pd.DataFrame(allinpdata)

In [ ]:
print(len(inpdata))
inpdata = inpdata.drop_duplicates()
inpdata = inpdata[inpdata["col"] != "function"]
print(len(inpdata))
print(len(set(inpdata["col"])))
print(set(inpdata["col"]))

# unique values seen in each column

In [ ]:
inpdata.groupby("col").count().reset_index()["val"].describe(percentiles=[0.75,0.9,0.99])

In [ ]:
dff = df[df["col"] == "function"]
dff

In [ ]:
#pdf = pd.concat(allpreds)
pdf = df

In [ ]:
pdf.head(5)

In [ ]:
#pdf = pdf[pdf.col.isin(COLS_TO_TABLES)]

# TODO: need equivalent version for SCOPE;
## TODO: more important is #columns per input in a query; for this we need column --> table mapping

In [ ]:
pdf.groupby(["col", "qid"])["op"].nunique().reset_index().sort_values(by="op", ascending=False)["op"].describe(
                                    percentiles=[0.75,0.90,0.99])

In [ ]:
for col in set(pdf["col"]):
    if not col in COLS_TO_TABLES:
        print(col)

In [ ]:
pdf2 = pdf[pdf.table != "X"]
print(set(pdf2["table"]))
print(len(set(pdf2["table"])))

In [ ]:
pdf2.groupby(["table", "qid"])["col"].nunique().reset_index().sort_values(by="col", ascending=False)["col"].describe(
                                    percentiles=[0.75,0.90,0.99])

In [ ]:
pd.set_option('display.max_rows', 500)
pdf2.head(100)

In [ ]:
pdf2.groupby(["table", "qid"])["col"].nunique().reset_index().sort_values(by="col", ascending=False)

# TODO:

#### More columns being filtered (tails) on BUT within a column ---- fewer categorical values being hit
#### discrete_type: num vs string
#### cont dtypes: range vs leq vs geq

In [ ]:
pdf.groupby("op").count()["qid"]